In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

from torch.utils.data import DataLoader, Dataset, Subset

/home/neiro/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import shutil
import random

from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold

In [3]:
from tqdm.auto import tqdm

In [5]:
mean=[0.485, 0.456, 0.406] # это среднее и стандартное отклонение всего датасета (обычно imagenet), на котором обучали большую сеть
std=[0.229, 0.224, 0.225]

transform_train = transforms.Compose([
         transforms.Resize(1024),
         #transforms.RandomRotation(30),
         transforms.RandomHorizontalFlip(),
         transforms.CenterCrop(1024),
         transforms.ToTensor(),
         transforms.Normalize(mean=mean,
                              std=std),
])

In [6]:
root_path = ''
datasetFolderName=root_path+'data'
sourceFiles=[]
classLabels=['0', '1', '3']
X=[]
Y=[]

train_path=datasetFolderName+'/train/'
validation_path=datasetFolderName+'/validation/'

In [7]:
def transferBetweenFolders(source, dest, splitRate): 
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', 
                               splitRate)

def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score

In [8]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
            if(folderName==classLabels[i]):
                Y.append(i)

In [9]:
# Organize file names and class labels in X and Y variables
for i in range(len(classLabels)):
    prepareNameWithLabels(classLabels[i])

In [10]:
X=np.asarray(X)
Y=np.asarray(Y)

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
def train(net, loss_fn, optimizer, train_loader,val_loader, n_epoch=10):
    best_metrics = 0

    for epoch in range(n_epoch):
        print(f'Epoch {epoch + 1}')
        train_dataiter = iter(train_loader)
        for i, batch in enumerate(tqdm(train_dataiter)):
            X_batch, y_batch = batch

            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()

            y_pred = net(X_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            metrics = []
            for batch in val_loader:
                x, y = batch
                x = x.to(device)
                y = y.to(device)
                y_pred = net(x)

                y_true = y.detach().cpu().numpy() 
                y_pred = np.argmax(y_pred.detach().cpu().numpy(), axis=1)
                f1_batch = f1_score(y_true, y_pred, average='macro')
                metrics.append(f1_batch)
            
            metrics = np.mean(np.array(metrics))
            # если стало лучше - сохраняем на диск и обновляем лучшую метрику
            if metrics > best_metrics: 
                print('New best model with test f1 macro:', metrics)
                torch.save(net.state_dict(), './best_model.pt')
                best_metrics = metrics
            if metrics == 1.0:
                break

    return net

In [13]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [14]:
vgg16 = models.vgg16(pretrained=True)

vgg16.classifier = nn.Sequential(*list(vgg16.classifier.children()))[:-1]

class New_VGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg16 = vgg16
        # for param in self.vgg16.features.parameters():
        #     param.requires_grad = False
        self.fc = nn.Sequential(nn.Linear(4096, 100),
                                nn.Linear(100, 3))
    
    def forward(self, x):
        x = self.vgg16(x)
        x = self.fc(x)
        return x
    
net = New_VGG16().to(device)

lr = 5e-4
loss_fn = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(net.parameters(), lr=lr)
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)

In [15]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
            if(Y_val[eachIndex]==i):
                classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
    
    train_dataset = datasets.ImageFolder(datasetFolderName+'/train/', transform=transform_train)
    val_dataset = datasets.ImageFolder(datasetFolderName+'/validation/', transform=transform_train)
    
    batch_size = 2

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=2)
    
    image_datasets = {'train': train_dataset, 'val': val_dataset}
    dataloaders = {'train': train_loader, 'val': val_loader}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = ['0','1','3']
    
    net = train(net, loss_fn, optimizer, train_loader, val_loader, n_epoch=25)

Results for fold 1
Epoch 1


100%|██████████| 200/200 [01:43<00:00,  1.92it/s]


New best model with test f1 macro: 0.8396624472573837
Epoch 2


100%|██████████| 200/200 [01:43<00:00,  1.93it/s]


New best model with test f1 macro: 0.8860759493670886
Epoch 3


...

New best model with test f1 macro: 0.9746835443037974
Epoch 2


100%|██████████| 200/200 [01:40<00:00,  1.99it/s]


New best model with test f1 macro: 1.0
Results for fold 3
Epoch 1


100%|██████████| 200/200 [01:40<00:00,  2.00it/s]


New best model with test f1 macro: 1.0


In [16]:
testdata_path = 'datatest'

transform_test = transforms.Compose([
         transforms.Resize(1024),
         transforms.CenterCrop(1024),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406], # это среднее и стандартное отклонение всего датасета (обычно imagenet), на котором обучали большую сеть
                              std=[0.229, 0.224, 0.225]),
])

test_data = datasets.ImageFolder(testdata_path, transform=transform_test)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=2)


pred = []
with torch.no_grad():
    for batch in test_loader:
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        y_pred = net(x)

        y_pred = np.argmax(y_pred.detach().cpu().numpy(), axis=1).tolist()
        pred.extend(y_pred)
    
pred2 = [3 if x==2 else x for x in pred]

import pandas as pd
sample = pd.read_csv("sample.csv")
sample['class'] = pred2

import os
files = []
 
for filename in os.listdir('datatest/testset'):
        if filename[filename.rfind(".") + 1:] in ['jpg', 'jpeg', 'png']:
            files.append(filename.split(".")[0])
files.sort()

sample['id'] = files
sample['id'] = sample['id'].astype(int)
sample = sample.sort_values(by=['id'])
sample.to_csv('sample73.csv', index=False)